#### Reading the data we prepared:

In [ ]:
import pandas as pd, numpy as np
np.random.seed(42)

DATA_PATH = '../cleaned_Airline_Delay_Data.csv'
df = pd.read_csv(DATA_PATH)
X = df.drop(columns=['arr_del15'])
y = df['arr_del15']

In [6]:
# I will split X and y into training and test sets according to years as this is time-series data. 
# this datasets spans 10 years so we train with 6, validate with 2 and test with 2 years of data.
train_mask = X['year'] <= 2019
X_train, y_train = X[train_mask], y[train_mask]

test_mask = X['year'] >= 2022
X_test, y_test = X[test_mask], y[test_mask]

val_mask = (X['year'] < 2022) & (X['year'] > 2019)
X_val, y_val = X[val_mask], y[val_mask]

#### In the previous notebook, I kept many features, but it’s often **better to select the most predictive ones rather than using all features blindly, as this reduces noise and improves efficiency**.
step 1: Remove collinearity

In [25]:
import feature_engine
from feature_engine.selection import DropCorrelatedFeatures
from sklearn.ensemble import RandomForestRegressor

tr = DropCorrelatedFeatures(variables=None, method='pearson', threshold=0.8)
Xt = tr.fit_transform(X_train)
tr.correlated_feature_sets_

[]

#### --> It seems none of our features are too **linearly** correlated.